# 加载模型

In [1]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="mistral-community/Mistral-7B-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistral-community/Mistral-7B-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistral-community/Mistral-7B-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [11]:
# prompt = "Hey, are you conscious? Can you talk to me?"
prompt = "please whrite a transformer code. Ok, "
inputs = tokenizer (prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"please whrite a transformer code. Ok, 1st of all, I'm not a programmer, so I'm not sure if I'm asking the right question. I'm trying to create a transformer that will take a string and return a string. The string will be a list of words, and the transformer will return a list of words. The transformer will take a string and return a string. The string will be a list of words,"

# 加载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("neuralwork/fashion-style-instruct")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'completion', 'context'],
        num_rows: 3193
    })
})

In [3]:
dataset["train"][0]

{'input': "I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.",
 'completion': 'Outfit Combination 1:\n- Top: Fitted white linen shirt\n- Bottom: Slim-fit beige chinos\n- Shoe: Brown leather loafers\n- Accessories: Brown woven belt, aviator sunglasses\n\nOutfit Combination 2:\n- Top: Light blue oxford button-down shirt\n- Bottom: Navy blue tailored trousers\n- Shoe: Tan leather brogues\n- Accessories: Navy blue patterned pocket square, silver wristwatch\n\nOutfit Combination 3:\n- Top: Light gray tailored blazer\n- Bottom: Dark wash denim jeans\n- Shoe: White canvas sneakers\n- Accessories: Black leather belt, silver pendant necklace\n\nOutfit Combination 4:\n- Top: Navy blue polo shirt\n- Bottom: Khaki shorts\n- Shoe: Brown leather sandals\n- Accessories: Navy blue baseball cap, woven brown bracelet\n\nOutfit Combination 5:\n- Top: Light pink dress shirt (with rolled-up sleeves)\n- Bottom: Charcoal gray dr

# 构建指令数据集

In [4]:
def format_instruction(sample) :
    return f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
{sample["input"]}
### Context:
{sample["context" ]}
### Response:
{sample["completion"] }
"""

In [5]:
sample = dataset["train"][0]
print(format_instruction(sample))

Tou are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.
### Context:
I'm going to a nature retreat.
### Response:
Outfit Combination 1:
- Top: Fitted white linen shirt
- Bottom: Slim-fit beige chinos
- Shoe: Brown leather loafers
- Accessories: Brown woven belt, aviator sunglasses

Outfit Combination 2:
- Top: Light blue oxford button-down shirt
- Bottom: Navy blue tailored trousers
- Shoe: Tan leather brogues
- Accessories: Navy blue patterned pocket square, silver wristwatch

Outfit Combination 3:
- Top: Light gray tailored blazer
- Bottom: Dark wash denim jeans
- Shoe: White canvas sneakers
- Accessories: Black leather belt, silver pendant necklace

Outfit Combi

# 定义量化参数

In [6]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 加载预训练模型

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-v0.1"
# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# model_id = "mistral-community/Mistral-7B-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
    )

model.config.use_cache=False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# 加载分词器

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id) 
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# 设置PEFT参数

In [9]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj"
        "up_proj",
        "down_proj"
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05, 
    task_type="CAUSAL_LM",
)

# 构建用于训练的PeftModel

In [10]:
from peft import prepare_model_for_kbit_training, get_peft_model

# prepare model for training

model = prepare_model_for_kbit_training(model)
model = get_peft_model (model, peft_config)

In [11]:
type(model)

peft.peft_model.PeftModelForCausalLM

# 打印模型中可训练参数的数量

In [12]:
def print_trainable_parameters(model):
    # Prints the number of trainable parameters in the model.
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel() 
        if param. requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params/all_param}")
    
# get frozen us trainable model param statistics
print_trainable_parameters(model)

trainable params: 27262976 || all params: 3779334144 || trainable%: 0.7213698223345028


# 设置用于训练的参数

In [13]:
from transformers import TrainingArguments

model_args = TrainingArguments(
    output_dir="mistral-7b-style",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

# 模型微调

In [14]:
from trl import SFTTrainer

max_seq_length=2048

# supervised Fine-Tuning Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=model_args,
)

Generating train split: 0 examples [00:00, ? examples/s]

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
trainer.train()

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.811100
20,0.666700
30,0.647600
40,0.620600
50,0.593000
60,0.580700
70,0.580400
80,0.570000
90,0.556400
100,0.557000


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=396, training_loss=0.5178189042842749, metrics={'train_runtime': 5807.9037, 'train_samples_per_second': 0.546, 'train_steps_per_second': 0.068, 'total_flos': 2.7734154551702323e+17, 'train_loss': 0.5178189042842749, 'epoch': 2.99})

In [16]:
trainer.save_model()